# Tokenizer

In [1]:
import json

In [2]:
data = []
with open('dump_small_clean.jsonln', 'r', encoding="utf8") as file:
    for line in file:
        data.append(json.loads(line))

In [3]:
 import nltk

In [85]:
from nltk import word_tokenize

In [8]:
import re


def minusculas(tokens):
    return [token.lower() for token in tokens]

# def remove_digitos(tokens):
#     return [token for token in tokens if re.fullmatch('[^\d]*', token)]

def pega_palavras(tokens):
    return [token for token in tokens if re.fullmatch('\w+', token)]
    
def limpa_tokens(tokens):
    tokens = minusculas(tokens)
#     tokens = remove_digitos(tokens)
    tokens = pega_palavras(tokens)    
    return tokens

In [9]:
from tqdm import tqdm

all_words = []
for item in tqdm(data):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = limpa_tokens(tokens)
    all_words += tokens

100%|███████████████████████████████████████████████████████████████████████████| 11225/11225 [00:32<00:00, 345.53it/s]


In [10]:
len(all_words)

3164393

In [11]:
all_words_clean = limpa_tokens(all_words)

In [12]:
len(all_words_clean)

3164393

In [13]:
def stopwords(word_list):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    return [i for i in word_list if i not in stopwords]
stopwords(all_words_clean)

['alexandre',
 'é',
 'prenome',
 'popular',
 'língua',
 'portuguesa',
 'é',
 'cognato',
 'nome',
 'alexander',
 'língua',
 'inglesa',
 'países',
 'lusófonos',
 'pessoas',
 'chamadas',
 'alexandre',
 'normalmente',
 'apelidadas',
 'alex',
 'origem',
 'nome',
 'deriva',
 'latim',
 'alexander',
 'romanização',
 'nome',
 'grego',
 'αλέξανδρος',
 'aléksandros',
 'etimologicamente',
 'nome',
 'composto',
 'verbo',
 'ἀλέξειν',
 'aléksein',
 'defender',
 'substantivo',
 'ἀνδρός',
 'andrós',
 'genitivo',
 'ἀνήρ',
 'anēr',
 'homem',
 'assim',
 'pode',
 'ser',
 'traduzido',
 'protetor',
 'humanidade',
 'termo',
 'tipo',
 'raro',
 'composto',
 'tatpurusha',
 'invertido',
 'modificante',
 'segunda',
 'posição',
 'tatpurusha',
 'cognato',
 'sânscrito',
 'sendo',
 'nararakṣa',
 'cf',
 'ramayana',
 'equivalente',
 'exato',
 'sânscrito',
 'rakṣinara',
 'pie',
 'composto',
 'gasto',
 'tipo',
 'terpsimbrotos',
 'cujo',
 'significado',
 'original',
 'protege',
 'homens',
 'primeiro',
 'registro',
 'conhec

In [14]:
from collections import Counter
word_counts = Counter(stopwords(all_words_clean))

In [15]:
word_counts_list = list(word_counts.items())

In [16]:
word_counts_list_sorted = sorted(word_counts_list, key=lambda x: (-x[1], x[0]))

In [17]:
vocab = word_counts_list_sorted[:10000] #10000 mais frequentes

In [18]:
vocab = dict(vocab) #Vocabulario de palavras

In [19]:
vocab

{'categoria': 28333,
 'é': 23656,
 'bgcolor': 10007,
 'the': 7568,
 'br': 6309,
 'ser': 4873,
 'of': 4553,
 'anos': 4459,
 'ligações': 4251,
 'externas': 4215,
 'brasil': 4069,
 'nbsp': 3992,
 'cidade': 3982,
 'estado': 3878,
 'rio': 3858,
 'sobre': 3426,
 'ano': 3348,
 'estados': 3229,
 'grande': 3105,
 'onde': 3012,
 'janeiro': 3009,
 'mount': 2937,
 'lemmon': 2910,
 'º': 2870,
 'primeira': 2848,
 'história': 2828,
 'durante': 2805,
 'paulo': 2785,
 'após': 2747,
 'unidos': 2745,
 'peak': 2694,
 'kitt': 2685,
 'primeiro': 2681,
 'spacewatch': 2679,
 'align': 2676,
 'parte': 2652,
 'dois': 2594,
 'filmes': 2587,
 'nome': 2516,
 'width': 2513,
 'jogos': 2504,
 'futebol': 2486,
 'fff': 2483,
 'família': 2457,
 'região': 2441,
 'and': 2400,
 'nova': 2383,
 'segundo': 2333,
 'in': 2332,
 'eee': 2331,
 'outros': 2306,
 'grupo': 2293,
 'sendo': 2286,
 'sul': 2281,
 'dia': 2271,
 'área': 2184,
 'série': 2161,
 'ainda': 2129,
 'portugal': 2126,
 'fafafa': 2112,
 'final': 2110,
 'ver': 2067,
 

In [77]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
LOWERCASE_OTHERS = ['ç', 'á', 'â', 'é', 'è', 'ó', 'ò', 'í', 'ì', 'ú', 'ù', ]  # etc.
LETTERS = LOWERCASE + LOWERCASE_OTHERS


def edit1(text):
    words = []
    
    # Fase 1: as remoçoes.
    for p in range(len(text)):
        new_word = text[:p] + text[p + 1:]
        if len(new_word) > 0:
            words.append(new_word)
        
    # Fase 2: as adições.
    for p in range(len(text) + 1):
        for c in LETTERS:
            new_word = text[:p] + c + text[p:]
            words.append(new_word)
    
    # Fase 3: as substituições.
    for p in range(len(text)):
        orig_c = text[p]
        for c in LETTERS:
            if orig_c != c:
                new_word = text[:p] + c + text[p + 1:]
                words.append(new_word)
    
    return set(words)

def edit2(text):
    words1 = edit1(text)
    words2 = set()
    for w in words1:
        candidate_words2 = edit1(w)
        candidate_words2 -= words1
        words2.update(candidate_words2)
    words2 -= set([text])
    return words2


In [79]:

def candidates(word):
    if word in vocab:
        candidatos = [word]
    else:
        candidatos = []
    candidatos += \
        [w for w in edit1(word) if w in vocab] \
        + [w for w in edit2(word) if w in vocab] \
        + [word] #Perguntar por que precisa
    return candidatos

In [80]:
candidates("alexandre")

['alexandre', 'alexandra', 'alejandro', 'alexandria', 'alexander', 'alexandre']

In [73]:
def P(word, N=sum(vocab.values())): 
        return vocab[word] / N

In [74]:
def correction(word):
    return max(candidates(word), key=P)

In [84]:
correction("alexand")

KeyError: 'alexand'